<a href="https://colab.research.google.com/github/GaetanAm/Master-1-Machine-Learning/blob/main/notebooks/TD4bis/Exercice_2_3_AMIENS_AMADIEU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXERCICE 2 : data undersampling

## Load the dataset in your notebook and check if there are any missing values

In [1]:
# Importer les bibliothèques nécessaires
from google.colab import files
import pandas as pd

# Télécharger le fichier localement
uploaded = files.upload()


Saving creditcard.csv to creditcard.csv


In [3]:
# Charger le dataset à partir du fichier téléchargé
data = pd.read_csv(list(uploaded.keys())[0])

# Vérifier les premières lignes
print("Aperçu du dataset :")
print(data.head())

# Vérifier les valeurs manquantes
print("\nValeurs manquantes dans chaque colonne :")
print(data.isnull().sum())

Aperçu du dataset :
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26 

No missing value, we have a proper dataset

## Create variables X and y, where X will contain the 30 features and y will be the target “Class”

In [5]:
# Définir X (caractéristiques) et y (cible)
X = data.iloc[:, :-1]  # Toutes les colonnes sauf la dernière (Class)
y = data['Class']      # Colonne cible

print(f"\nDimensions des données : X = {X.shape}, y = {y.shape}")



Dimensions des données : X = (284807, 30), y = (284807,)


## Split the data into a training set and a test set with a test set of 20%.
You need to import train_test_split from sklearn.model selection. By setting the stratify argument to data[”Class”], you’re instructing the
train_test_split function to preserve the class proportions in both the training and testing sets. This ensures
that both classes are represented in a balanced manner. You can check the number of values present in a set
with the method value_counts() to ensure that both classes are well represented in y_train and y_test.

In [6]:
from sklearn.model_selection import train_test_split

# Diviser les données en train et test avec stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Vérifier la répartition des classes
print("\nRépartition des classes dans y_train :")
print(y_train.value_counts(normalize=True))
print("\nRépartition des classes dans y_test :")
print(y_test.value_counts(normalize=True))



Répartition des classes dans y_train :
Class
0    0.998271
1    0.001729
Name: proportion, dtype: float64

Répartition des classes dans y_test :
Class
0    0.99828
1    0.00172
Name: proportion, dtype: float64


Cette répartition initiale démontre un fort déséquilibre des classes. Le sous-échantillonnage permet de travailler sur un dataset équilibré pour entraîner le modèle.

## Create your undersampled data using the resample method from sklearn.utils to keep the same number of samples for each target “Class”.
Be careful : resampling should be applied only to the training set to avoid
data leakage.

Data leakage occurs when information from the testing set is inadvertently used to train the model.

This can lead to overfitting and inflated performance metrics, as the model becomes too tailored to the specific examples in the testing set.


In [8]:
from sklearn.utils import resample

# Combiner X_train et y_train pour faciliter la manipulation
train_data = pd.concat([X_train, y_train], axis=1)

# Séparer les classes majoritaire et minoritaire
majority_class = train_data[train_data['Class'] == 0]
minority_class = train_data[train_data['Class'] == 1]

# Sous-échantillonnage de la classe majoritaire
majority_downsampled = resample(
    majority_class,
    replace=False,                # Échantillonnage sans remplacement
    n_samples=len(minority_class),  # Même nombre que la classe minoritaire
    random_state=42
)

# Combiner les deux classes
undersampled_data = pd.concat([majority_downsampled, minority_class])

# Répartition après sous-échantillonnage
print("\nRépartition des classes après sous-échantillonnage :")
print(undersampled_data['Class'].value_counts())

# Redéfinir X_train et y_train
X_train_undersampled = undersampled_data.iloc[:, :-1]
y_train_undersampled = undersampled_data['Class']



Répartition des classes après sous-échantillonnage :
Class
0    394
1    394
Name: count, dtype: int64


Les classes sont équilibrées : 394 exemples dans chaque classe.

## Train a Logistic Regression model on the original data and another Logistic Regression model on the undersampled data.
We will then compare the performance of these two models to see if our undersampled data allowed better results.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Modèle sur les données originales
model_original = LogisticRegression(max_iter=200, random_state=42)
model_original.fit(X_train, y_train)
y_pred_original = model_original.predict(X_test)

print("\nRapport de classification (données originales) :")
print(classification_report(y_test, y_pred_original))

# Modèle sur les données sous-échantillonnées
model_undersampled = LogisticRegression(max_iter=200, random_state=42)
model_undersampled.fit(X_train_undersampled, y_train_undersampled)
y_pred_undersampled = model_undersampled.predict(X_test)

print("\nRapport de classification (données sous-échantillonnées) :")
print(classification_report(y_test, y_pred_undersampled))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c


Rapport de classification (données originales) :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.76      0.69      0.72        98

    accuracy                           1.00     56962
   macro avg       0.88      0.85      0.86     56962
weighted avg       1.00      1.00      1.00     56962


Rapport de classification (données sous-échantillonnées) :
              precision    recall  f1-score   support

           0       1.00      0.95      0.98     56864
           1       0.03      0.92      0.06        98

    accuracy                           0.95     56962
   macro avg       0.52      0.94      0.52     56962
weighted avg       1.00      0.95      0.97     56962



## Evaluate both models by comparing the accuracy, precision, recall and F1-score metrics.

### Précision globale (accuracy) :

Le modèle original a une précision parfaite, mais cela est trompeur dans un contexte de données déséquilibrées.
Le modèle sous-échantillonné a une précision légèrement inférieure à 95 %, mais il offre une meilleure détection des fraudes.

### Rappel (classe fraude) :

Le rappel passe de 0.69 à 0.92, ce qui est crucial pour capturer un maximum de fraudes.

### F1-score (classe fraude) :

Bien que le rappel augmente avec le sous-échantillonnage, la précision faible tire le F1-score vers le bas.

# EXERCICE 3 : Random Oversampling Technique


Simple random oversampling involves randomly duplicating samples from the minority class to balance the
dataset.

When to Use:

  - Moderate unbalanced class: When the majority class doesn’t significantly outnumber the minority class.

  - Small datasets: When the dataset is relatively small and increasing the sample size can improve model performance.


The first steps are the same: load the dataset, check missing values, train/test split data. Let’s continue to the
part where we check if our sets possess all possible values with value counts().

## Create your oversampled data using the resample method from sklearn.utils to keep the same number ofsamples for each value of the target “Class”.


In [7]:
from sklearn.utils import resample

# Combiner X_train et y_train pour faciliter la manipulation
train_data = pd.concat([X_train, y_train], axis=1)

# Séparer les classes majoritaire et minoritaire
majority_class = train_data[train_data['Class'] == 0]
minority_class = train_data[train_data['Class'] == 1]

# Oversampling de la classe minoritaire
minority_oversampled = resample(
    minority_class,
    replace=True,                 # Oversampling avec remplacement
    n_samples=len(majority_class), # Même nombre que la classe majoritaire
    random_state=42
)

# Combiner les deux classes
oversampled_data = pd.concat([majority_class, minority_oversampled])

# Répartition après oversampling
print("\nRépartition des classes après oversampling :")
print(oversampled_data['Class'].value_counts())

# Redéfinir X_train et y_train
X_train_oversampled = oversampled_data.iloc[:, :-1]
y_train_oversampled = oversampled_data['Class']



Répartition des classes après oversampling :
Class
0    227451
1    227451
Name: count, dtype: int64


## Train a Logistic Regression model on the original data and another Logistic Regression model on the undersampled data. We will then compare the performance of these two models to see if our undersampled data allowed better results.

Evaluate the 2 models by printing the accuracy, precision, recall and F1-score metrics.

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
# Modèle sur les données originales
model_original = LogisticRegression(max_iter=200, random_state=42)
model_original.fit(X_train, y_train)
y_pred_original = model_original.predict(X_test)

print("\nRapport de classification (données originales) :")
print(classification_report(y_test, y_pred_original))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Rapport de classification (données originales) :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.76      0.69      0.72        98

    accuracy                           1.00     56962
   macro avg       0.88      0.85      0.86     56962
weighted avg       1.00      1.00      1.00     56962



In [12]:
# Modèle sur les données sur-échantillonnées
model_oversampled = LogisticRegression(max_iter=200, random_state=42)
model_oversampled.fit(X_train_oversampled, y_train_oversampled)
y_pred_oversampled = model_oversampled.predict(X_test)

print("\nRapport de classification (données sur-échantillonnées) :")
print(classification_report(y_test, y_pred_oversampled))



Rapport de classification (données sur-échantillonnées) :
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     56864
           1       0.05      0.91      0.09        98

    accuracy                           0.97     56962
   macro avg       0.52      0.94      0.54     56962
weighted avg       1.00      0.97      0.98     56962



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Make a quick analysis of the performance metrics of both undersampled and oversampled data (you should get approximatively the same conclusions).

### Données originales :
Bonne précision globale, mais faible rappel pour la classe fraude.
Peu adapté à un système où la détection des fraudes est essentielle.

En détails:

- Classe 0 (légitime) :

 - Précision et rappel parfaits : le modèle est fortement biaisé en faveur de la classe majoritaire.


- Classe 1 (fraude) :
 - Le rappel est 0.69, ce qui signifie que 31 % des fraudes ne sont pas détectées.

 - La précision de 0.76 montre que des transactions légitimes sont mal classées comme frauduleuses.

### Données sur-échantillonnées :
Excellente capacité à détecter les fraudes (rappel élevé), mais beaucoup de faux positifs.

Approche utile si le rappel est prioritaire

En détails:
- Classe 0 (légitime) :
 - La précision reste élevée (1.00), mais le rappel diminue légèrement (0.97).
- Classe 1 (fraude) :
 - Rappel amélioré à 0.91, capturant presque toutes les fraudes.
 - Précision très faible (0.05), ce qui génère de nombreux faux positifs.